In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# =======================
# HYPERPARAMETERS
# =======================
num_epochs = 4            # Jumlah epoch (berapa kali seluruh dataset dilatih)
batch_size = 4            # Jumlah gambar per batch
learning_rate = 0.001     # Learning rate optimizer

# =======================
# DATA TRANSFORM
# =======================
transform = transforms.Compose([
    transforms.ToTensor(),                        # Mengubah gambar menjadi tensor (C, H, W)
    transforms.Normalize(                         # Normalisasi pixel ke range [-1, 1]
        (0.5, 0.5, 0.5),                          # Mean untuk 3 channel RGB
        (0.5, 0.5, 0.5)                           # Std untuk 3 channel RGB
    )
])

# =======================
# DATASET CIFAR-10
# =======================
train_dataset = torchvision.datasets.CIFAR10(
    root='./data',           # Folder penyimpanan dataset
    train=True,              # Dataset training
    download=True,           # Download jika belum ada
    transform=transform      # Terapkan transform
)

test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,             # Dataset testing
    download=True,
    transform=transform
)

# =======================
# DATALOADER
# =======================
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,   # Ambil 4 gambar per batch
    shuffle=True             # Acak data tiap epoch
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False            # Tidak perlu diacak saat testing
)

# Label kelas CIFAR-10
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# =======================
# CNN MODEL
# =======================
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 6, 5)      # Conv layer: input 3 channel RGB → 6 feature maps
        self.pool = nn.MaxPool2d(2, 2)       # Max pooling 2x2 (downsampling)
        self.conv2 = nn.Conv2d(6, 16, 5)     # Conv layer kedua: 6 → 16 feature maps

        self.fc1 = nn.Linear(16*5*5, 120)    # Fully connected layer
        self.fc2 = nn.Linear(120, 84)        # Hidden FC layer
        self.fc3 = nn.Linear(84, 10)         # Output layer (10 kelas CIFAR-10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # Conv1 → ReLU → Pool
        x = self.pool(F.relu(self.conv2(x))) # Conv2 → ReLU → Pool
        x = x.view(-1, 16*5*5)               # Flatten tensor ke bentuk vektor
        x = F.relu(self.fc1(x))              # Fully connected + ReLU
        x = F.relu(self.fc2(x))              # Fully connected + ReLU
        x = self.fc3(x)                      # Output logits (tanpa softmax)
        return x

# =======================
# MODEL, LOSS, OPTIMIZER
# =======================
model = ConvNet().to(device)                # Kirim model ke CPU/GPU

criterion = nn.CrossEntropyLoss()           # Loss untuk klasifikasi multi-class
optimizer = torch.optim.SGD(
    model.parameters(),                     # Parameter yang akan diupdate
    lr=learning_rate                        # Learning rate
)

# =======================
# TRAINING LOOP
# =======================
n_total_steps = len(train_loader)            # Jumlah batch per epoch

for epoch in range(num_epochs):              # Loop epoch
    for i, (images, labels) in enumerate(train_loader):

        images = images.to(device)           # Kirim gambar ke device
        labels = labels.to(device)           # Kirim label ke device

        outputs = model(images)              # Forward pass
        loss = criterion(outputs, labels)    # Hitung loss

        optimizer.zero_grad()                # Reset gradient
        loss.backward()                      # Backpropagation
        optimizer.step()                     # Update bobot

        if (i+1) % 2000 == 0:                 # Print progress
            print(f'Epoch [{epoch+1}/{num_epochs}], '
                  f'Step [{i+1}/{n_total_steps}], '
                  f'Loss: {loss.item():.4f}')

print('Finished Training')

# =======================
# EVALUATION
# =======================
with torch.no_grad():                        # Nonaktifkan gradient
    n_correct = 0
    n_samples = 0

    n_class_correct = [0 for _ in range(10)] # Benar per kelas
    n_class_samples = [0 for _ in range(10)] # Total per kelas

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)              # Forward pass
        _, predicted = torch.max(outputs, 1) # Ambil kelas dengan logit terbesar

        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):          # Hitung akurasi per kelas
            label = labels[i]
            pred = predicted[i]
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')


# ============================================================
# RINGKASAN:
# Kode ini membangun dan melatih Convolutional Neural Network (CNN)
# untuk klasifikasi gambar CIFAR-10 (10 kelas). Data diproses dalam
# batch, dilewatkan ke layer convolution + pooling untuk ekstraksi
# fitur, kemudian ke fully connected layer untuk klasifikasi.
# Model dilatih menggunakan SGD dan CrossEntropyLoss, lalu diuji
# untuk mendapatkan akurasi total dan akurasi per kelas.
# ============================================================

Epoch [1/4], Step [2000/12500], Loss: 2.3360
Epoch [1/4], Step [4000/12500], Loss: 2.3029
Epoch [1/4], Step [6000/12500], Loss: 2.2754
Epoch [1/4], Step [8000/12500], Loss: 2.2935
Epoch [1/4], Step [10000/12500], Loss: 2.2955
Epoch [1/4], Step [12000/12500], Loss: 2.2734
Epoch [2/4], Step [2000/12500], Loss: 2.3114
Epoch [2/4], Step [4000/12500], Loss: 2.2009
Epoch [2/4], Step [6000/12500], Loss: 2.5225
Epoch [2/4], Step [8000/12500], Loss: 2.3410
Epoch [2/4], Step [10000/12500], Loss: 2.0884
Epoch [2/4], Step [12000/12500], Loss: 2.0791
Epoch [3/4], Step [2000/12500], Loss: 2.2530
Epoch [3/4], Step [4000/12500], Loss: 1.6076
Epoch [3/4], Step [6000/12500], Loss: 1.9960
Epoch [3/4], Step [8000/12500], Loss: 1.8377
Epoch [3/4], Step [10000/12500], Loss: 2.1582
Epoch [3/4], Step [12000/12500], Loss: 0.9024
Epoch [4/4], Step [2000/12500], Loss: 1.6744
Epoch [4/4], Step [4000/12500], Loss: 1.1883
Epoch [4/4], Step [6000/12500], Loss: 2.3448
Epoch [4/4], Step [8000/12500], Loss: 1.2132
Epoc

In [ ]:
# ============================
# CATATAN PENTING: CNN (Convolutional Neural Network)
# ============================

# 1. CNN adalah arsitektur neural network yang DIRANCANG KHUSUS
#    untuk data spasial seperti gambar (image).
#    CNN memanfaatkan struktur 2D (tinggi x lebar x channel).

# 2. Alur umum CNN:
#    Input Image
#      → Convolutional Layer (ekstraksi fitur)
#      → Activation (ReLU)
#      → Pooling (downsampling)
#      → (diulang beberapa kali)
#      → Flatten
#      → Fully Connected Layer
#      → Output (klasifikasi)

# 3. Convolutional Layer (nn.Conv2d):
#    - Menerima input dalam bentuk (channels, height, width)
#    - Parameter utama:
#        in_channels  : jumlah channel input (RGB = 3, grayscale = 1)
#        out_channels : jumlah filter / feature maps (bebas ditentukan)
#        kernel_size  : ukuran filter (misal 3x3, 5x5)
#    - out_channels menentukan JUMLAH FITUR yang dipelajari

# 4. Output channel (jumlah filter) dan kernel size TIDAK FIX:
#    - Dapat ditentukan bebas oleh perancang model
#    - Dipilih berdasarkan:
#        • kompleksitas data
#        • kapasitas model
#        • resource komputasi
#    - Semakin banyak filter → semakin banyak fitur → lebih berat komputasi

# 5. Activation Function (ReLU):
#    - Diterapkan setelah convolutional layer
#    - Menghilangkan nilai negatif
#    - Memberikan non-linearitas
#    - ReLU bekerja pada SETIAP neuron / pixel hasil convolution
#    - Tanpa ReLU, CNN hanya menjadi kombinasi linear

# 6. Pooling Layer (MaxPool2d):
#    - Berfungsi untuk downsampling (mengurangi resolusi)
#    - Contoh: kernel=2, stride=2 → ukuran menjadi setengah
#    - Mengurangi komputasi dan meningkatkan robustness
#    - Pooling BUKAN layer pembelajaran (tidak punya parameter)

# 7. Mengapa Convolution dilakukan SEBELUM Fully Connected?
#    - Conv layer → mengekstraksi fitur lokal & hierarkis
#    - FC layer → mengambil keputusan berdasarkan fitur global
#    - CNN memisahkan:
#        • feature extraction (Conv)
#        • classification (FC)

# 8. Flatten:
#    - Mengubah feature map (C x H x W) menjadi vektor 1D
#    - Wajib sebelum masuk Fully Connected Layer

# 9. Fully Connected Layer (nn.Linear):
#    - Menggabungkan semua fitur hasil convolution
#    - Bertindak sebagai classifier
#    - Biasanya tetap menggunakan ReLU (kecuali layer output)

# 10. Output Layer:
#     - Tidak menggunakan ReLU
#     - Menghasilkan logits (skor kelas)
#     - Logits akan diproses oleh:
#         • Softmax (multi-class)
#         • Sigmoid (binary classification)

# 11. CNN unggul karena:
#     - Mempertahankan struktur spasial gambar
#     - Menggunakan weight sharing
#     - Parameter lebih efisien dibanding MLP
#     - Lebih akurat dan stabil untuk image classification

# ============================
# RINGKASAN:
# CNN memisahkan proses ekstraksi fitur (Conv + ReLU + Pool)
# dan pengambilan keputusan (Fully Connected),
# sehingga sangat efektif untuk klasifikasi gambar.
# ============================
